## About data
#### Data used is breast cancer RNAseq dataset download from Firehose (https://gdac.broadinstitute.org/). This data has already been normalized 

In [1]:
# load libraries
library(randomForest)
library(class)
library(dplyr)

randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘dplyr’


The following object is masked from ‘package:randomForest’:

    combine


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
#Download dataset from Firehose
url_data <- "http://gdac.broadinstitute.org/runs/stddata__2016_01_28/data/BRCA/20160128/gdac.broadinstitute.org_BRCA.Merge_rnaseqv2__illuminahiseq_rnaseqv2__unc_edu__Level_3__RSEM_genes_normalized__data.Level_3.2016012800.0.0.tar.gz"
dest_file <- "~/Desktop/git_repo/Predict-disease-from-gene-expression-data/data.gz"
download.file(url_data, dest_file)

In [2]:
# read dataset
data_set <- read.table("BRCA.rnaseqv2__illuminahiseq_rnaseqv2__unc_edu__Level_3__RSEM_genes_normalized__data.data.txt", 
            header = F, fill = T)
head(data_set)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V1205,V1206,V1207,V1208,V1209,V1210,V1211,V1212,V1213,V1214
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Hybridization,REF,TCGA-3C-AAAU-01A-11R-A41B-07,TCGA-3C-AALI-01A-11R-A41B-07,TCGA-3C-AALJ-01A-31R-A41B-07,TCGA-3C-AALK-01A-11R-A41B-07,TCGA-4H-AAAK-01A-12R-A41B-07,TCGA-5L-AAT0-01A-12R-A41B-07,TCGA-5L-AAT1-01A-12R-A41B-07,TCGA-5T-A9QA-01A-11R-A41B-07,⋯,TCGA-UL-AAZ6-01A-11R-A41B-07,TCGA-UU-A93S-01A-21R-A41B-07,TCGA-V7-A7HQ-01A-11R-A33J-07,TCGA-W8-A86G-01A-21R-A36F-07,TCGA-WT-AB41-01A-11R-A41B-07,TCGA-WT-AB44-01A-11R-A41B-07,TCGA-XX-A899-01A-11R-A36F-07,TCGA-XX-A89A-01A-11R-A36F-07,TCGA-Z7-A8R5-01A-42R-A41B-07,TCGA-Z7-A8R6-01A-11R-A41B-07
2,gene_id,normalized_count,normalized_count,normalized_count,normalized_count,normalized_count,normalized_count,normalized_count,normalized_count,normalized_count,⋯,normalized_count,normalized_count,normalized_count,normalized_count,normalized_count,normalized_count,normalized_count,normalized_count,normalized_count,
3,?|100130426,0.0000,0.0000,0.9066,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,⋯,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,
4,?|100133144,16.3644,9.2659,11.6228,12.0894,6.8468,3.9889,0.0000,1.4644,15.3396,⋯,4.3126,0.0000,5.5624,0.0000,0.0000,14.3858,22.3240,2.2638,6.8865,
5,?|100134869,12.9316,17.3790,9.2294,11.0799,14.4298,13.6090,10.5949,8.9958,14.3935,⋯,10.8828,3.0792,14.3711,6.3091,3.2580,21.4409,27.2744,7.2933,24.7795,
6,?|10357,52.1503,69.7553,154.2974,143.8643,84.2128,114.2572,115.9984,107.5628,116.3870,⋯,136.1288,29.9974,128.3151,53.6278,42.2643,137.7756,64.1427,85.0461,167.5511,


In [3]:
# drop first 2 columns
data_set <- data_set %>%
             select(-c(1,2)) # drop hybridization and REF columns
head(data_set)

,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,⋯,V1205,V1206,V1207,V1208,V1209,V1210,V1211,V1212,V1213,V1214
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,TCGA-3C-AAAU-01A-11R-A41B-07,TCGA-3C-AALI-01A-11R-A41B-07,TCGA-3C-AALJ-01A-31R-A41B-07,TCGA-3C-AALK-01A-11R-A41B-07,TCGA-4H-AAAK-01A-12R-A41B-07,TCGA-5L-AAT0-01A-12R-A41B-07,TCGA-5L-AAT1-01A-12R-A41B-07,TCGA-5T-A9QA-01A-11R-A41B-07,TCGA-A1-A0SB-01A-11R-A144-07,TCGA-A1-A0SD-01A-11R-A115-07,⋯,TCGA-UL-AAZ6-01A-11R-A41B-07,TCGA-UU-A93S-01A-21R-A41B-07,TCGA-V7-A7HQ-01A-11R-A33J-07,TCGA-W8-A86G-01A-21R-A36F-07,TCGA-WT-AB41-01A-11R-A41B-07,TCGA-WT-AB44-01A-11R-A41B-07,TCGA-XX-A899-01A-11R-A36F-07,TCGA-XX-A89A-01A-11R-A36F-07,TCGA-Z7-A8R5-01A-42R-A41B-07,TCGA-Z7-A8R6-01A-11R-A41B-07
2,normalized_count,normalized_count,normalized_count,normalized_count,normalized_count,normalized_count,normalized_count,normalized_count,normalized_count,normalized_count,⋯,normalized_count,normalized_count,normalized_count,normalized_count,normalized_count,normalized_count,normalized_count,normalized_count,normalized_count,
3,0.0000,0.9066,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,⋯,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,
4,9.2659,11.6228,12.0894,6.8468,3.9889,0.0000,1.4644,15.3396,9.5178,10.0370,⋯,4.3126,0.0000,5.5624,0.0000,0.0000,14.3858,22.3240,2.2638,6.8865,
5,17.3790,9.2294,11.0799,14.4298,13.6090,10.5949,8.9958,14.3935,11.3241,4.4426,⋯,10.8828,3.0792,14.3711,6.3091,3.2580,21.4409,27.2744,7.2933,24.7795,
6,69.7553,154.2974,143.8643,84.2128,114.2572,115.9984,107.5628,116.3870,60.2630,153.1452,⋯,136.1288,29.9974,128.3151,53.6278,42.2643,137.7756,64.1427,85.0461,167.5511,


In [4]:
# The top 2 rows of data_set contains additional header informations that are not needed for downstream analysis. 
mrna_data <- data_set %>% 
             slice(-c(1,2)) #drop top 2 rows
head(mrna_data)

,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,⋯,V1205,V1206,V1207,V1208,V1209,V1210,V1211,V1212,V1213,V1214
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0.0000,0.9066,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,⋯,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,
2,9.2659,11.6228,12.0894,6.8468,3.9889,0.0000,1.4644,15.3396,9.5178,10.0370,⋯,4.3126,0.0000,5.5624,0.0000,0.0000,14.3858,22.3240,2.2638,6.8865,
3,17.3790,9.2294,11.0799,14.4298,13.6090,10.5949,8.9958,14.3935,11.3241,4.4426,⋯,10.8828,3.0792,14.3711,6.3091,3.2580,21.4409,27.2744,7.2933,24.7795,
4,69.7553,154.2974,143.8643,84.2128,114.2572,115.9984,107.5628,116.3870,60.2630,153.1452,⋯,136.1288,29.9974,128.3151,53.6278,42.2643,137.7756,64.1427,85.0461,167.5511,
5,563.8934,1360.8341,865.5358,766.3830,807.7431,1108.3945,1420.5021,657.2812,977.9175,1084.3274,⋯,2886.3965,1721.8816,697.6744,1245.2681,1877.4180,652.7559,722.7208,1140.2801,1003.5668,
6,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,⋯,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,


In [5]:
# Patient IDs
pat_IDs <- data_set %>% 
           slice_head()
pat_IDs

V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,⋯,V1205,V1206,V1207,V1208,V1209,V1210,V1211,V1212,V1213,V1214
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
TCGA-3C-AAAU-01A-11R-A41B-07,TCGA-3C-AALI-01A-11R-A41B-07,TCGA-3C-AALJ-01A-31R-A41B-07,TCGA-3C-AALK-01A-11R-A41B-07,TCGA-4H-AAAK-01A-12R-A41B-07,TCGA-5L-AAT0-01A-12R-A41B-07,TCGA-5L-AAT1-01A-12R-A41B-07,TCGA-5T-A9QA-01A-11R-A41B-07,TCGA-A1-A0SB-01A-11R-A144-07,TCGA-A1-A0SD-01A-11R-A115-07,⋯,TCGA-UL-AAZ6-01A-11R-A41B-07,TCGA-UU-A93S-01A-21R-A41B-07,TCGA-V7-A7HQ-01A-11R-A33J-07,TCGA-W8-A86G-01A-21R-A36F-07,TCGA-WT-AB41-01A-11R-A41B-07,TCGA-WT-AB44-01A-11R-A41B-07,TCGA-XX-A899-01A-11R-A36F-07,TCGA-XX-A89A-01A-11R-A36F-07,TCGA-Z7-A8R5-01A-42R-A41B-07,TCGA-Z7-A8R6-01A-11R-A41B-07


## Data preprocessing
#### Created a diagnostic class - 0 for normal and 1 for tumor. the infromation about label class can be found in the IDs dataframe. For #### example in ID: TCGA-3C-AAAU-01A-11R-A41B.07 the sample type is indicated in the 4th group ("01A"). Tumor types range from 01 - 09 and #### from 10 - 19 for normal types and 20 - 29 for control. Therefore, it is neccesary to extract from the IDs which sample type is normal #### or not.

In [6]:
# Extract sample types from IDs
samp_class <- lapply(as.list(t(pat_IDs)), function(t) substr(unlist(strsplit(t, "-"))[4], 1, 2))
samp_class

[[1]]
[1] "01"

[[2]]
[1] "01"

[[3]]
[1] "01"

[[4]]
[1] "01"

[[5]]
[1] "01"

[[6]]
[1] "01"

[[7]]
[1] "01"

[[8]]
[1] "01"

[[9]]
[1] "01"

[[10]]
[1] "01"

[[11]]
[1] "01"

[[12]]
[1] "01"

[[13]]
[1] "01"

[[14]]
[1] "01"

[[15]]
[1] "01"

[[16]]
[1] "01"

[[17]]
[1] "01"

[[18]]
[1] "01"

[[19]]
[1] "01"

[[20]]
[1] "01"

[[21]]
[1] "01"

[[22]]
[1] "01"

[[23]]
[1] "01"

[[24]]
[1] "01"

[[25]]
[1] "01"

[[26]]
[1] "01"

[[27]]
[1] "01"

[[28]]
[1] "01"

[[29]]
[1] "01"

[[30]]
[1] "01"

[[31]]
[1] "01"

[[32]]
[1] "01"

[[33]]
[1] "01"

[[34]]
[1] "01"

[[35]]
[1] "01"

[[36]]
[1] "01"

[[37]]
[1] "01"

[[38]]
[1] "01"

[[39]]
[1] "01"

[[40]]
[1] "01"

[[41]]
[1] "01"

[[42]]
[1] "01"

[[43]]
[1] "01"

[[44]]
[1] "01"

[[45]]
[1] "01"

[[46]]
[1] "01"

[[47]]
[1] "01"

[[48]]
[1] "01"

[[49]]
[1] "01"

[[50]]
[1] "01"

[[51]]
[1] "01"

[[52]]
[1] "01"

[[53]]
[1] "01"

[[54]]
[1] "01"

[[55]]
[1] "01"

[[56]]
[1] "01"

[[57]]
[1] "01"

[[58]]
[1] "01"

[[59]]
[1] "01"

[[60]]
[1] "01"

[[61]]
[1] "01"

[[62]]
[1] "01"

[[63]]
[1] "01"

[[64]]
[1] "01"

[[65]]
[1] "01"

[[66]]
[1] "01"

[[67]]
[1] "01"

[[68]]
[1] "01"

[[69]]
[1] "01"

[[70]]
[1] "01"

[[71]]
[1] "01"

[[72]]
[1] "01"

[[73]]
[1] "01"

[[74]]
[1] "01"

[[75]]
[1] "01"

[[76]]
[1] "01"

[[77]]
[1] "01"

[[78]]
[1] "01"

[[79]]
[1] "01"

[[80]]
[1] "01"

[[81]]
[1] "01"

[[82]]
[1] "01"

[[83]]
[1] "01"

[[84]]
[1] "01"

[[85]]
[1] "01"

[[86]]
[1] "01"

[[87]]
[1] "01"

[[88]]
[1] "01"

[[89]]
[1] "01"

[[90]]
[1] "01"

[[91]]
[1] "01"

[[92]]
[1] "01"

[[93]]
[1] "01"

[[94]]
[1] "01"

[[95]]
[1] "01"

[[96]]
[1] "01"

[[97]]
[1] "01"

[[98]]
[1] "01"

[[99]]
[1] "01"

[[100]]
[1] "01"

[[101]]
[1] "01"

[[102]]
[1] "01"

[[103]]
[1] "01"

[[104]]
[1] "01"

[[105]]
[1] "01"

[[106]]
[1] "01"

[[107]]
[1] "01"

[[108]]
[1] "01"

[[109]]
[1] "01"

[[110]]
[1] "01"

[[111]]
[1] "01"

[[112]]
[1] "01"

[[113]]
[1] "01"

[[114]]
[1] "01"

[[115]]
[1] "01"

[[116]]
[1] "01"

[[117]]
[1] "01"

[[118]]
[1] "01"

[[119]]
[1] "01"

[[120]]
[1] "01"

[[121]]
[1] "01"

[[122]]
[1] "01"

[[123]]
[1] "01"

[[124]]
[1] "01"

[[125]]
[1] "01"

[[126]]
[1] "11"

[[127]]
[1] "01"

[[128]]
[1] "01"

[[129]]
[1] "11"

[[130]]
[1] "01"

[[131]]
[1] "01"

[[132]]
[1] "11"

[[133]]
[1] "01"

[[134]]
[1] "01"

[[135]]
[1] "11"

[[136]]
[1] "01"

[[137]]
[1] "01"

[[138]]
[1] "11"

[[139]]
[1] "01"

[[140]]
[1] "11"

[[141]]
[1] "01"

[[142]]
[1] "11"

[[143]]
[1] "01"

[[144]]
[1] "01"

[[145]]
[1] "01"

[[146]]
[1] "01"

[[147]]
[1] "01"

[[148]]
[1] "01"

[[149]]
[1] "01"

[[150]]
[1] "01"

[[151]]
[1] "01"

[[152]]
[1] "01"

[[153]]
[1] "01"

[[154]]
[1] "01"

[[155]]
[1] "01"

[[156]]
[1] "01"

[[157]]
[1] "01"

[[158]]
[1] "01"

[[159]]
[1] "01"

[[160]]
[1] "01"

[[161]]
[1] "01"

[[162]]
[1] "01"

[[163]]
[1] "01"

[[164]]
[1] "01"

[[165]]
[1] "01"

[[166]]
[1] "01"

[[167]]
[1] "01"

[[168]]
[1] "01"

[[169]]
[1] "01"

[[170]]
[1] "01"

[[171]]
[1] "01"

[[172]]
[1] "01"

[[173]]
[1] "01"

[[174]]
[1] "01"

[[175]]
[1] "01"

[[176]]
[1] "01"

[[177]]
[1] "01"

[[178]]
[1] "01"

[[179]]
[1] "01"

[[180]]
[1] "01"

[[181]]
[1] "01"

[[182]]
[1] "01"

[[183]]
[1] "01"

[[184]]
[1] "01"

[[185]]
[1] "01"

[[186]]
[1] "01"

[[187]]
[1] "01"

[[188]]
[1] "01"

[[189]]
[1] "01"

[[190]]
[1] "01"

[[191]]
[1] "01"

[[192]]
[1] "01"

[[193]]
[1] "01"

[[194]]
[1] "01"

[[195]]
[1] "01"

[[196]]
[1] "01"

[[197]]
[1] "01"

[[198]]
[1] "01"

[[199]]
[1] "01"

[[200]]
[1] "01"

[[201]]
[1] "01"

[[202]]
[1] "01"

[[203]]
[1] "01"

[[204]]
[1] "01"

[[205]]
[1] "01"

[[206]]
[1] "01"

[[207]]
[1] "01"

[[208]]
[1] "01"

[[209]]
[1] "01"

[[210]]
[1] "01"

[[211]]
[1] "01"

[[212]]
[1] "01"

[[213]]
[1] "01"

[[214]]
[1] "01"

[[215]]
[1] "01"

[[216]]
[1] "01"

[[217]]
[1] "01"

[[218]]
[1] "01"

[[219]]
[1] "01"

[[220]]
[1] "01"

[[221]]
[1] "01"

[[222]]
[1] "01"

[[223]]
[1] "01"

[[224]]
[1] "01"

[[225]]
[1] "01"

[[226]]
[1] "01"

[[227]]
[1] "01"

[[228]]
[1] "01"

[[22

In [7]:
diag_class <- as.data.frame(samp_class)
diag_class
dim(diag_class)

X.01.,X.01..1,X.01..2,X.01..3,X.01..4,X.01..5,X.01..6,X.01..7,X.01..8,X.01..9,⋯,X.01..1083,X.01..1084,X.01..1085,X.01..1086,X.01..1087,X.01..1088,X.01..1089,X.01..1090,X.01..1091,X.01..1092
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
01,01,01,01,01,01,01,01,01,01,⋯,01,01,01,01,01,01,01,01,01,01


[1]    1 1212

In [8]:
#count number of each type
table(unlist(diag_class))


  01   06   11 
1093    7  112 

In [9]:
diag_class[diag_class[1,] < 10] <- 1
diag_class[diag_class[1,] > 10] <- 0

In [10]:
#count of tumor samples and normal samples
table(unlist(diag_class))


   0    1 
 112 1100 